# Sistema de detección de enlaces spam

---

Queremos implementar un sistema que sea capaz de detectar automáticamente si una página web contiene spam o no basándonos en su URL

In [1]:
import pandas as pd

---

## Paso 1: Carga del conjunto de datos

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv")
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


---

## Paso 2: Preprocesa los enlaces

Utiliza lo visto en este módulo para transformar los datos para compatibilizarlos con el modelo que queremos entrenar. Segmenta las URLs en partes según sus signos de puntuación, elimina las stopwords, lematiza, etcétera.

Asegúrate de dividir convenientemente el conjunto de datos en train y test como hemos visto en lecciones anteriores.

### 2.1 Transformación categórica a numérica

Transformamos la categoría `is_spam` en valores numéricos (0 y 1), ya que este, como la mayoría de los modelos, no trabajan con variables de clase categóricas:

In [3]:
# Transformamos True y False en 1 y 0. Para indicar si es spam o no

df['is_spam'] = df['is_spam'].apply(lambda x: 1 if x == True else 0).astype(int)
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


### 2.2 Eliminación de valores repetidos

Podemos contar fácilmente cuántos casos de cada clase tenemos para analizar si el conjunto de datos está equilibrado o no:

In [4]:
row, col = df.shape

print(f"{row} datos y {col} categorías")

print(f"Spam: {len(df.loc[df['is_spam'] == 1])}")

print(f"No Spam: {len(df.loc[df['is_spam'] == 0])}")



2999 datos y 2 categorías
Spam: 696
No Spam: 2303


>Como podemos ver no está equilibrado porque tenemos 696 mensajes de spam y 2303 mensajes de no spam.

Debemos eliminar también los duplicados, si los hubiera:

In [5]:
# Obtain the number of duplicate values in our dataset
def procesado_duplicados(dataframe):

    duplicados = dataframe.duplicated().sum()
    print(f"The number of duplicate values in the dataset is {duplicados}")

    if duplicados > 0:

        print("Después del procesado tenemos...")
        
        dataframe = dataframe.drop_duplicates()

        dataframe = dataframe.reset_index(inplace = False, drop = True)

        final_duplicados = dataframe.duplicated().sum()

        print(f"The number of duplicate values in the dataset is {final_duplicados}")

    return dataframe 

df = procesado_duplicados(df)

The number of duplicate values in the dataset is 630
Después del procesado tenemos...
The number of duplicate values in the dataset is 0


En este caso vemos que se han eliminado más de 600 registros repetidos.


---


### 2.3 Procesamiento del texto

Para poder entrenar el modelo es necesario aplicar antes un proceso de transformación al texto. Comenzamos transformando el texto a minúsculas y eliminando signos de puntuación y caracteres especiales:

In [9]:
# Library which can be used to work with Regular Expressions
# https://www.w3schools.com/python/python_regex.asp

import regex as re

def preprocess_text(text):

    # Eliminar cualquier carácter que no sea una letra (a-z) o un espacio en blanco ( )

    text = re.sub(r'[^a-z ]',#  La construcción [^...] indica negación en las expresiones regulares,
                             # lo que significa "coincide con cualquier cosa que no esté en este conjunto
                   " ", # " ": Esta es la cadena de reemplazo. Cualquier carácter que coincida con el patrón será reemplazado por un espacio.
                     text) 


    # Eliminar espacios en blanco
    text = re.sub(r'\s+[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\^[a-zA-Z]\s+', " ", text)

    # Reducir espacios en blanco múltiples a uno único
    text = re.sub(r'\s+', " ", text.lower())

    # Eliminar tags
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)

    return text.split()

df["url"] = df["url"].apply(preprocess_text)
df.head()

,url,is_spam
0,"[https, briefingday, us, list, manage, com, un...",1
1,"[https, www, hvper, com]",1
2,"[https, briefingday, com, v, i]",1
3,"[https, briefingday, com, m, commentform]",0
4,"[https, briefingday, com, fan]",1
